In [103]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the correct column names
column_names = ['ET', 'V', 'N', 'SA', 'IA', 'RL', 'RE', 'P', 'FX', 'FY', 'FZ', 'MX', 'MZ',
                'NFX', 'NFY', 'RST', 'TSTI', 'TSTC', 'TSTO', 'AMBTMP', 'SR']

# Define the Pacejka model function
def pacejka_model(alpha, B, C, D, E, F):
    return -(D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F))

# Function to calculate the local slope using the difference method
def calculate_local_slope_pacejka(x_values, y_values):
    slopes = np.diff(y_values) / np.diff(x_values)
    # For plotting, return an array of the same length as x_values (hence, use np.nan for the last point)
    return np.append(slopes, np.nan)

# Function to load and prepare the dataframe
def load_and_prepare_dataframe(file_path):
    try:
        df = pd.read_csv(file_path, delim_whitespace=True, names=column_names, on_bad_lines='skip')
        print(f"Successfully loaded {file_path}")
    except Exception as e:
        print(f"Failed to load {file_path}. Error: {e}")
        return None

    df = df.iloc[2:].reset_index(drop=True)
    if len(df) > 0:
        return df
    else:
        print(f"File {file_path} has zero length after preprocessing.")
        return None

# Function to process data and generate side-by-side plots
def process_and_save_side_by_side_plots(file_paths, output_directory):
    for file_path in file_paths:
        # Load and prepare the dataframe
        df = load_and_prepare_dataframe(file_path)
        if df is None:
            continue

        print(f"Processing file: {file_path}")

        # Convert columns to numeric and clean data
        df['FY'] = pd.to_numeric(df['FY'], errors='coerce') * -1 / 1000  # Convert FY to kN
        df['SA'] = pd.to_numeric(df['SA'], errors='coerce')
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(subset=['FY', 'SA'], inplace=True)

        # Fit the Pacejka model to the data
        x_data = df['SA']
        y_data = df['FY']
        try:
            popt, _ = curve_fit(pacejka_model, x_data, y_data, p0=[0.5, 1.2, max(y_data), 1, 0], maxfev=175000)
            fitted_x = np.linspace(x_data.min(), x_data.max(), 100)
            fitted_curve = pacejka_model(fitted_x, *popt)
            slope_curve = calculate_local_slope_pacejka(fitted_x, fitted_curve)
        except Exception as e:
            print(f"Error fitting curve for {file_path}: {e}")
            continue

        # Set dynamic axis limits for the second plot based on the data
        y_max_with_gap = y_data.max() * 1.05  # 5% gap added to FY
        x_min = x_data.min()
        x_max = x_data.max()
        x_min_with_gap = x_min * 1.05 if x_min < 0 else x_min * 0.95  # Add a negative gap if x_min < 0
        x_max_with_gap = x_max * 1.05  # 5% gap added to the max slip angle

        # Create subplots: 1 row, 2 columns
        fig = make_subplots(rows=1, cols=2, subplot_titles=("Lateral Force vs. Slip Angle", "Instantaneous Cornering Stiffness vs Slip Angle"))

        # Add scatter plot of actual data (FY vs SA)
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='markers', name='Raw Data (FY vs SA)'), row=1, col=1)

        # Add fitted Pacejka curve (FY vs SA)
        fig.add_trace(go.Scatter(x=fitted_x, y=fitted_curve, mode='lines', name='Pacejka Fit'), row=1, col=1)

        # Add slope plot (Slope of Pacejka Fit vs SA)
        fig.add_trace(go.Scatter(x=fitted_x, y=slope_curve, mode='lines', name='Instantaneous Cornering Stiffness vs Slip Angle'), row=1, col=2)

        # Set layout for both plots
        fig.update_layout(
            title_text=f'Lateral Force and Slope of Pacejka Fit for {os.path.basename(file_path)}',
            autosize=True,
            width=1400,
            height=600,
            margin=dict(l=50, r=50, b=65, t=90)
        )

        # Set individual plot ranges and labels
        fig.update_xaxes(title_text='Slip Angle (deg)', range=[0, 13], row=1, col=1)  # Left plot: fixed range 0-13
        fig.update_yaxes(title_text='Lateral Force (kN)', range=[0, y_max_with_gap], row=1, col=1)

        fig.update_xaxes(title_text='Slip Angle (deg)', range=[x_min_with_gap, x_max_with_gap], row=1, col=2)  # Right plot: dynamic range with negative gap
        fig.update_yaxes(title_text='Instantaneous Cornering Stiffness (kN/deg)', row=1, col=2)

        # Save plot to output directory
        output_path = os.path.join(output_directory, f"{os.path.basename(file_path)}_fy_and_slope_vs_sa.html")
        fig.write_html(output_path)
        print(f"Saved plot to {output_path}")

In [ ]:
# Example usage on a list of file paths
file_paths = [
    '/Users/peytonboone/Desktop/feb/fall2024/instantaneousCorneringStiffness/B2356raw11.dat',
    '/Users/peytonboone/Desktop/feb/fall2024/instantaneousCorneringStiffness/B2356raw15.dat',
    '/Users/peytonboone/Desktop/feb/fall2024/instantaneousCorneringStiffness/B2356raw4.dat',
]

# Specify the output directory for saving the HTML files
output_directory = "/Users/peytonboone/Desktop/feb/fall2024/instantenousCorneringStiffness"

# Call the function to process and save 3D graphs as HTML files using Plotly
process_and_save_side_by_side_plots(file_paths, output_directory)